<img src="https://docs.dask.org/en/latest/_images/dask_horizontal.svg" align="right" width="30%"/>

<center>
    <h1>
Dask Tutorial for PyHEP 2022
    </h1>
</center>

Dask is a pure Python library for parallel and distributed computing designed to scale up workflows in the PyData ecosystem.

The two main components of Dask:

- The **collections library(ies)** (sometimes called "Dask core") listed here in alphabetical order:
  - `dask.array`: chunked NumPy
  - `dask.bag`: partitioned Python iterables
  - `dask.dataframe`: partitioned Pandas
  - `dask.delayed`: custom algorithms
- The **execution engines** (task schedulers)
  - The distributed engine is its own project (`distributed`, sometimes called "Dask Distributed")
 

<div style="text-align: center;">
  <img src="https://docs.dask.org/en/stable/_images/dask-overview.svg" align="center" width="70%"/>
</div>

There are also a number of other projects in the Dask ecosystem that leverage both upstream components.

First Example (using `dask.delayed`)
------------------------------------

We'll start with a simple `dask.delayed` example that covers _a lot_ of how Dask works:

In [ ]:
import dask
from dask.delayed import delayed

def inc(x):
    return x + 1

inc = delayed(inc)

In [ ]:
inc(7)

Notice that this just creates a `Delayed` object.

In [ ]:
eight = inc(7)

We have to ask Dask to determine the result via `compute()`

In [ ]:
eight.compute()

We can start to construct a more complex task graph by chaining function calls:

In [ ]:
@delayed
def inc(x):
    return x + 1

@delayed
def add(x, y):
    return x + y

In [ ]:
five = add(inc(1), inc(2))

In [ ]:
five.compute()

We can inspect the complete task graph to see how dask accomplishing computing the result of the collection:

In [ ]:
delayed_task_graph = five.dask.to_dict()

In [ ]:
for i, (k, v) in enumerate(delayed_task_graph.items()):
    if i != 0:
        print("\n")
    print("The key (label) of a task:   ", k)
    print("The task itself (Lisp S-exp):", v)

There is a much better method of inspection! (`visualize()`)

In [ ]:
five.visualize(engine="cytoscape")

Second Example
--------------

Let's take a look at an example that illustrates something closer to a real workflow: reading and operating on files to produce a histogram. Our example will have two steps:

1. Load an uproot TTree by file and tree name
2. Calculate something from information in the file
3. Histogram the calculation

We'll look at the workflow while leveraging Dask, and compare to a workflow without Dask

In [ ]:
import uproot
import awkward as ak
import hist
import time
from skhep_testdata import data_path
paths = [data_path("uproot-Zmumu.root")] * 4

In [ ]:
@delayed
def read_tree(file_name, tree_name):
    time.sleep(1)  # faking making the file larger
    return uproot.open(file_name)[tree_name]

@delayed
def calculation(tree):
    arrs = tree.arrays()
    return abs(arrs.E1 - arrs.E2)
    
@delayed
def histo(data, bins, range):
    h = hist.Hist(hist.axis.Regular(bins=bins, start=range[0], stop=range[1], name="abs(E1-E2)"))
    h.fill(data)
    return h

In [ ]:
histos = []
for p in paths:
    tree = read_tree(p, "events")
    calc = calculation(tree)
    h = histo(calc, 20, (0, 200))
    histos.append(h)

In [ ]:
histos

In [ ]:
sum(histos).visualize(engine="cytoscape")

In [ ]:
sum(histos).compute()

In [ ]:
%%timeit
sum(histos).compute()

Now without Dask:

In [ ]:
def s_read_tree(file_name, tree_name):
    time.sleep(1)  # faking making the file larger
    return uproot.open(file_name)[tree_name]

def s_calculation(tree):
    arrs = tree.arrays()
    return abs(arrs.E1 - arrs.E2)
    
def s_histo(data, bins, range):
    h = hist.Hist(hist.axis.Regular(bins=bins, start=range[0], stop=range[1], name="abs(E1-E2)"))
    h.fill(data)
    return h

In [ ]:
%%timeit
s_histos = []
for p in paths:
    tree = s_read_tree(p, "events")
    calc = s_calculation(tree)
    h = histo(calc, 20, (0, 200))
    s_histos.append(h)
sum(s_histos)

dask.array
==========

While `dask.delayed` is incredibly flexible and can turn almost any Python function into a node in a task graph, the other collection libraries are designed to provide task graph creation as a near drop in replacement to existing PyData libraries. The NumPy API is meant to be recreated with `dask.array`. Arrays in `dask.array` are chunked and lazily evaluated NumPy arrays. The data nodes in a task graph are just NumPy arrays: Dask doesn't create a new array computation kernel library. 

<center>
<img src="https://docs.dask.org/en/stable/_images/dask-array.svg" width="50%">
</center>

In [ ]:
import numpy as np
import dask.array as da

In [ ]:
a1 = np.ones((10,))

In [ ]:
a1.sum()

In [ ]:
a1[:5].sum() + a1[5:].sum()

In [ ]:
a2 = da.ones((10,), chunks=5)

In [ ]:
a2

In [ ]:
a2.sum()

In [ ]:
a3 = a2 + 1
dask.compute(a2.sum(), a3)

Chaining together function calls with `dask.array` is very similar to what we did with `dask.delayed`: it simply builds up the task graph. However, now we get to use the ubiquitous NumPy API.

In [ ]:
x = da.ones((25000, 25000), chunks=(5000, 5000))
x

In [ ]:
y = x + x.T
z = da.mean(y[::2, :5000:2], axis=1)

In [ ]:
z.visualize(engine="cytoscape", color="order")

In [ ]:
z.visualize(engine="cytoscape", color="order", optimize_graph=True)

In [ ]:
z

In [ ]:
z.compute()

dask.dataframe
==============

The NumPy/dask.array relationship is mirrored for Pandas with dask.dataframe. DataFrames(Series) in dask.dataframe are partitioned and lazily evalualated Pandas DataFrames(Series). The data nodes in a task graph are pandas objects.

<center>
<img src="https://docs.dask.org/en/stable/_images/dask-dataframe.svg" width="35%">
</center>

In [ ]:
from dask.datasets import timeseries

In [ ]:
ddf = timeseries()

In [ ]:
ddf

In [ ]:
ddf.groupby("name")[["x", "y"]].mean().compute()

distributed
===========

So far we've been using the default execution engine at each `.compute()` call: we're using all discovered threads. This parallelizes computation on a laptop, for example.

The distributed engine uses a client + cluster model, where the cluster is composed of a scheduler and any number of workers.

<center>
<img src="images/distributed.png" width=60%"/>
</center>

In [ ]:
from distributed import Client, LocalCluster
cluster = LocalCluster()

In [ ]:
cluster

In [ ]:
client = Client(cluster)

In [ ]:
client

The ecosystem of clusters is large!

- [dask-jobqueue](https://jobqueue.dask.org/en/latest/index.html) (HTCondor, Slurm, PBS, and more)
- [dask-cloudprovider](https://docs.dask.org/en/latest/deploying-cloud.html)
- [dask-kubernetes](https://kubernetes.dask.org/en/latest/)
- [dask-gateway](https://gateway.dask.org/)
- ...

Simplified example:

```python
from distributed import Client
from dask_jobqueue import HTCondorCluster
cluster = HTCondorCluster(memory="2GB")
cluster.scale(10)
client = Client(cluster)

discovery = client.compute(my_big_task)
```

**Note for folks using binder**: the dashboard part of the live tutorial will be unavailable

In [ ]:
z.compute()

Adding to the ecosystem!
========================

[dask-awkward](https://github.com/ContinuumIO/dask-awkward) and [dask-histogram](https://github.com/dask-contrib/dask-histogram): Native support for awkward arrays and boost-histogram objects in Dask. To showcase these projects we'll work out a small example

**Note**: dask-awkward depends on Awkward Array version 2!

In [ ]:
import dask_awkward as dak
import dask_histogram as dh
import awkward._v2 as ak

In [ ]:
ds = dak.from_parquet(
    "s3://ddavistemp/higgs_pq/*.parquet",
    storage_options={"anon": True},
)

In [ ]:
ds

In [ ]:
two_muons = ds.muons[dak.num(ds.muons, axis=1) == 2]

In [ ]:
leading_pt = dak.max(two_muons.pt, axis=1).map_partitions(ak.fill_none, 0)

In [ ]:
cut = leading_pt > 25

In [ ]:
second_pt = dak.min(two_muons[cut].pt, axis=1)

In [ ]:
h = dh.histogram(second_pt, bins=20, range=(0, 70), histogram=True)

In [ ]:
hist.Hist(h.compute()).plot1d()

In [ ]:
h.visualize()

In [ ]:
h.visualize(optimize_graph=True)

Final Thoughts
==============
We've just scratched the surface of the Dask universe, but hopefully you've learn something! More resources for Dask in general:

- [The Dask documentation](https://docs.dask.org/en/stable/)
- [The official Dask Tutorial](https://github.com/dask/dask-tutorial) (multi hour tutorial)

The [dask-awkward project](https://github.com/ContinuumIO/dask-awkward) welcomes users willing to live a bit on the edge! Please stay tuned for follow up presentations showcasing more Dask <--> Scikit-HEP work.